In [17]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [18]:
#Parsing Training documents
L_train = pd.read_parquet('book_train.parquet/stock_id=0')
df_train = pd.read_csv('train.csv')


In [19]:
#now to create a function that works out the Weighted average price 
def Wap(df):
    # Formula: (BidPrice * AskSize + AskPrice * BidSize) / TotalSize
    a=df['bid_price1']*df['ask_size1']
    b=df['ask_price1']*df['bid_size1']
    c=df['ask_size1']+df['bid_size1']
    return (a+b)/c
L_train['wap'] = Wap(L_train)


In [20]:
def Log_rets(x):
    #This works out ln(current/previous price)
    return np.log(x).diff()

In [21]:
#Getting Log Returns 
L_train['log_returns'] = L_train.groupby('time_id')['wap'].transform(Log_rets)
L_train=L_train.dropna()


In [22]:
#Now to caluclate realised volatility
#I found HFT the mean should be considered 0 so i updated the formula to reflect th\t
def Realisedvol(x):
    return np.sqrt(np.sum(x**2))

df_features = L_train.groupby('time_id').agg(volatility_feature=('log_returns',Realisedvol)).reset_index()


In [23]:
#creating a final dataframe for stock 0
stock0_target = df_train[df_train['stock_id']==0]
df_final = pd.merge(stock0_target, df_features,on='time_id',how='inner')


In [ ]:
# Now to implement Machine Learning
X=df_final['volatility_feature']]
Y=df_final['target']

In [25]:
#Now split the data for training and testing
#traintestsplit test size is percent left for training and random states give the same randomness every time for testign

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
#initialise the model
model=lgb.LGBMRegressor()
model.fit(X_train,Y_train)
#Predictions in the test set
Y_pred = model.predict(X_test)
#Now the score Calculating
#Using RMS of percentage error
def rmspe(Y_true, Y_pred):
    return np.sqrt(np.mean(np.square((Y_true - Y_pred) / Y_true)))
score=rmspe(Y_test,Y_pred)

print(f'Final Model Score (RMSPE): {score}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000067 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 3064, number of used features: 1
[LightGBM] [Info] Start training from score 0.004053
Final Model Score (RMSPE): 0.35415977029204554
